In [4]:
import csv
import json
import unicodedata
import numpy as np
import networkx as nx

import requests
import pandas as pd
from pattern import web
import matplotlib.pyplot as plt
import re

from matplotlib import rcParams
from bs4 import BeautifulSoup

In [5]:
def find_str(s, char):
    index = 0
    if char in s:
        c = char[0]
        for ch in s:
            if ch == c:
                if s[index:index+len(char)] == char:
                    return index

            index += 1

    return -1

In [6]:
def get_poll_xml1_draft(poll_id):
    url = "http://www.hockey-reference.com/draft/NHL_%i_entry.html" % int(poll_id)
    return requests.get(url).text

In [7]:
def get_poll_xml1_player(poll_id):
    url = "http://www.hockey-reference.com" + poll_id

    return requests.get(url).text

In [8]:
def RepresentsInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

In [9]:
teams=['ANA','PHX','BOS','BUF','CGY','CAR','CHI','COL','CBJ','DAL','DET','EDM','FLA','LAK','MIN','MTL','NSH','NJD','NYI','NYR','OTT','PHI','PIT','SJS','STL','TBL','TOR','VAN','WSH',
'WPG','QUE','WIN','MDA','HAR','MNS','ATL','PHX']

In [11]:
for year in range(1989,2016):
    df_frames=[]
    matrix=[]
    print year
    for team in teams:
        url='http://www.hockey-reference.com/teams/'+team+'/'+str(year)+'_games.html'
        page=requests.get(url).text
        pagesoap=BeautifulSoup(page, "lxml")
        strsoap=str(pagesoap)
        posseStr=(find_str(strsoap, "Defensive Zone Faceoffs / (Offensive Zone Faceoffs + Defensive Zone Faceoffs),<br />that took place while on the ice"))

        posStr=(find_str(strsoap, "Awards</th>\n</tr>\n</thead>\n<tbody>"))
        posEnd=(find_str(strsoap, "</tr>\n</tfoot>\n</table>\n</div>\n</div>\n</div><!-- div.table_wrapper -->\n<!-- sr_mobile_1 -->"))
        posseEnd=(find_str(strsoap, " <h2>NHL Miscellaneous</h2>"))

        strlimitPos=strsoap[(posseStr+116):posseEnd]

        dates_game=[]
        for m in re.finditer('data-stat="date_game"', strlimitPos):
            if strlimitPos[(m.end()+40):(m.end()+44)].isdigit() and strlimitPos[(m.end()+45):(m.end()+47)].isdigit() and \
                strlimitPos[(m.end()+48):(m.end()+50)].isdigit():
                dates_game.append(strlimitPos[(m.end()+40):(m.end()+50)])




        num_games=[]
        for m in re.finditer('data-stat="games"', strlimitPos):
            if strlimitPos[(m.end()+13):(m.end()+15)].isdigit():
                num_games.append(strlimitPos[(m.end()+13):(m.end()+15)])
            elif strlimitPos[(m.end()+13):(m.end()+14)].isdigit():
                num_games.append(strlimitPos[(m.end()+13):(m.end()+14)])



        tm_agt=[]
        tm_for=[]
        for m in re.finditer('data-stat="opp_name"', strlimitPos):
            if strlimitPos[(m.end()+17):(m.end()+20)].isupper():
                tm_agt.append(strlimitPos[(m.end()+17):(m.end()+20)])
                tm_for.append(team)


        tm_loc_game=[]
        for m in re.finditer('data-stat="game_location"', strlimitPos):
            if strlimitPos[(m.end()+3):(m.end()+5)]=='td' or strlimitPos[(m.end()+4):(m.end()+6)]=='td':
                if strlimitPos[(m.end()+1)]=='@':
                    tm_loc_game.append('A')
                elif strlimitPos[(m.end()+1)]=='<':
                    tm_loc_game.append('H')


        tm_goals_pp_for=[]
        for m in re.finditer('data-stat="goals_pp"', strlimitPos):
            if strlimitPos[(m.end()+1):(m.end()+3)].isdigit():
                tm_goals_pp_for.append(strlimitPos[(m.end()+1):(m.end()+3)])
            elif strlimitPos[(m.end()+1):(m.end()+2)].isdigit():
                tm_goals_pp_for.append(strlimitPos[(m.end()+1):(m.end()+2)])

        tm_goals_sh_for=[]
        for m in re.finditer('data-stat="goals_sh"', strlimitPos):
            if strlimitPos[(m.end()+1):(m.end()+3)].isdigit():
                tm_goals_sh_for.append(strlimitPos[(m.end()+1):(m.end()+3)])
            elif strlimitPos[(m.end()+1):(m.end()+2)].isdigit():
                tm_goals_sh_for.append(strlimitPos[(m.end()+1):(m.end()+2)])


        tm_goals_agt_sh=[]
        for m in re.finditer('data-stat="goals_against_sh"', strlimitPos):
            if strlimitPos[(m.end()+1):(m.end()+3)].isdigit():
                tm_goals_agt_sh.append(strlimitPos[(m.end()+1):(m.end()+3)])
            elif strlimitPos[(m.end()+1):(m.end()+2)].isdigit():
                tm_goals_agt_sh.append(strlimitPos[(m.end()+1):(m.end()+2)])

        tm_goals_agt_pp=[]
        for m in re.finditer('data-stat="goals_against_pp"', strlimitPos):
            if strlimitPos[(m.end()+1):(m.end()+3)].isdigit():
                tm_goals_agt_pp.append(strlimitPos[(m.end()+1):(m.end()+3)])
            elif strlimitPos[(m.end()+1):(m.end()+2)].isdigit():
                tm_goals_agt_pp.append(strlimitPos[(m.end()+1):(m.end()+2)])

        opp_pen_min=[]
        for m in re.finditer('data-stat="pen_min_opp"', strlimitPos):
            if strlimitPos[(m.end()+1):(m.end()+3)].isdigit():
                opp_pen_min.append(strlimitPos[(m.end()+1):(m.end()+3)])
            elif strlimitPos[(m.end()+1):(m.end()+2)].isdigit():
                opp_pen_min.append(strlimitPos[(m.end()+1):(m.end()+2)])

        tm_shots_agt=[]
        for m in re.finditer('data-stat="shots_against"', strlimitPos):
            if strlimitPos[(m.end()+1):(m.end()+3)].isdigit():
                tm_shots_agt.append(strlimitPos[(m.end()+1):(m.end()+3)])
            elif strlimitPos[(m.end()+1):(m.end()+2)].isdigit():
                tm_shots_agt.append(strlimitPos[(m.end()+1):(m.end()+2)])

        tm_pen_min=[]
        for m in re.finditer('data-stat="pen_min"', strlimitPos):
            if strlimitPos[(m.end()+1):(m.end()+3)].isdigit():
                tm_pen_min.append(strlimitPos[(m.end()+1):(m.end()+3)])
            elif strlimitPos[(m.end()+1):(m.end()+2)].isdigit():
                tm_pen_min.append(strlimitPos[(m.end()+1):(m.end()+2)])

        tm_shots_for=[]
        for m in re.finditer('data-stat="shots"', strlimitPos):
            if strlimitPos[(m.end()+1):(m.end()+3)].isdigit():
                tm_shots_for.append(strlimitPos[(m.end()+1):(m.end()+3)])
            elif strlimitPos[(m.end()+1):(m.end()+2)].isdigit():
                tm_shots_for.append(strlimitPos[(m.end()+1):(m.end()+2)])

        cum_loss_for=[]
        for m in re.finditer('data-stat="losses"', strlimitPos):
            if strlimitPos[(m.end()+1):(m.end()+3)].isdigit():
                cum_loss_for.append(strlimitPos[(m.end()+1):(m.end()+3)])
            elif strlimitPos[(m.end()+1):(m.end()+2)].isdigit():
                cum_loss_for.append(strlimitPos[(m.end()+1):(m.end()+2)])

        cum_wins_for=[]
        for m in re.finditer('data-stat="wins"', strlimitPos):
            if strlimitPos[(m.end()+1):(m.end()+3)].isdigit():
                cum_wins_for.append(strlimitPos[(m.end()+1):(m.end()+3)])
            elif strlimitPos[(m.end()+1):(m.end()+2)].isdigit():
                cum_wins_for.append(strlimitPos[(m.end()+1):(m.end()+2)])

        gm_outcome=[]
        for m in re.finditer('data-stat="game_outcome"', strlimitPos):
            if strlimitPos[(m.end()+1):(m.end()+2)] == 'W' or strlimitPos[(m.end()+1):(m.end()+2)] == 'L' or strlimitPos[(m.end()+1):(m.end()+2)] == 'T':
                gm_outcome.append(strlimitPos[(m.end()+1):(m.end()+2)])


        tm_goals_agt=[]
        for m in re.finditer('data-stat="opp_goals"', strlimitPos):
            if strlimitPos[(m.end()+1):(m.end()+3)].isdigit():
                tm_goals_agt.append(strlimitPos[(m.end()+1):(m.end()+3)])
            elif strlimitPos[(m.end()+1):(m.end()+2)].isdigit():
                tm_goals_agt.append(strlimitPos[(m.end()+1):(m.end()+2)])


        tm_goals_for=[]
        for m in re.finditer('data-stat="goals"', strlimitPos):
            if strlimitPos[(m.end()+1):(m.end()+3)].isdigit():
                tm_goals_for.append(strlimitPos[(m.end()+1):(m.end()+3)])
            elif strlimitPos[(m.end()+1):(m.end()+2)].isdigit():
                tm_goals_for.append(strlimitPos[(m.end()+1):(m.end()+2)])


        tm_loss=[]
        for m in re.finditer('data-stat="game_streak"', strlimitPos):
            if strlimitPos[(m.end()+3):(m.end()+5)].isdigit():
                tm_loss.append(strlimitPos[(m.end()+1):(m.end()+5)])
            elif strlimitPos[(m.end()+3):(m.end()+4)].isdigit():
                tm_loss.append(strlimitPos[(m.end()+1):(m.end()+4)])

        matrix=[tm_for,num_games,dates_game,tm_agt,tm_loc_game,tm_goals_for,tm_goals_agt,gm_outcome,cum_wins_for,cum_loss_for,
                 tm_loss,tm_shots_for,tm_pen_min,tm_goals_pp_for,tm_goals_sh_for,tm_shots_agt,opp_pen_min,tm_goals_agt_pp,
               tm_goals_agt_sh]
        
        matrix_T=zip(*matrix)
        colnames = ['tm_for','num_games','dates_game','tm_agt','gm_loc','tm_goals_for','tm_goals_agt','gm_result','cum_wins_for','cum_loss_for','tm_loss','tm_shots_for','tm_pen_min','tm_goals_pp_for','tm_goals_sh_for','tm_shots_agt','opp_pen_min','tm_goals_agt_pp','tm_goals_agt_sh']
        
        new_df=pd.DataFrame(matrix_T,columns=colnames)
        df_frames.append(new_df)
    df_all=pd.concat(df_frames)
    df_all.to_csv('team_data_games_data_'+str(year)+'.csv')

1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
